# MPC Action Codes API

#### This tutorial provides information on how to use the Minor Planet Center's Action Codes API.

The Minor Planet Center's `Action Codes` service allows you to retrieve action codes that were sent when you originally submitted observations to the MPC.

**What are Action Codes?**

Action codes are automatically generated codes that are sent to the submitters of new NEO and new comet candidates as well as to the NEO/NEOPC follow-up reporters. These action codes can be used to:
 - Retract (delete) your submitted observations
 - Replace observations with corrected versions
 - Prove ownership of your submissions

If you've lost your action code, this API can help you retrieve it. **The action code will be emailed to the original submitter's address.**

The Action Codes API is a REST endpoint. You can send POST requests to:

    https://data.minorplanetcenter.net/api/action-codes/retrieve

In the examples below we use Python code to query the API.

Further information and documentation can be found at:
 - https://docs.minorplanetcenter.net/mpc-ops-docs/apis/action-codes/

# Import Packages
Here we import the standard Python packages needed to call the API.

In [ ]:
import requests
import json

# API Parameters

The Action Codes API accepts the following parameter:

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `label` | String | Yes | An identifier for your submission |

### Label Formats

The `label` parameter can be one of several formats:

| Format | Pattern | Example |
|--------|---------|--------|
| Tracklet Submission ID (trksub) | 1-8 alphanumeric/underscore | `ABC12345` |
| Track ID (trkid) | 8-12 alphanumeric/underscore | `ABCD12345678` |
| Submission Block ID | `YYYY-MM-DDTHH:MM:SS.mmm_xxxxxxxx_nn` | `2024-01-15T10:30:00.123_ABCD1234_01` |
| Submission ID | `YYYY-MM-DDTHH:MM:SS.mmm_xxxxxxxx` | `2024-01-15T10:30:00.123_ABCD1234` |

### Important Notes

1. **This is a POST request**, not GET (unlike most other MPC APIs)
2. **The action code is sent via email** to the original submitter's email address
3. The API response itself does not contain the action code

# How It Works

When you call this API with a valid submission label:

1. The MPC looks up the submission in their database
2. If found, they retrieve the original submitter's email address
3. The action code is sent to that email address
4. The API returns a confirmation (but not the code itself)

This ensures that only the original submitter can retrieve their action code.

# Basic Usage

Here's how to request an action code retrieval.

**Note:** Replace `your-submission-id` with your actual submission ID. 

<span style="color: red;">The response in this example will indicate a problem (because the supplied submission-ID,  `your-submission-id` is not a valid submission-ID).</span>


In [ ]:
# Example: Request action code for a submission
# Replace with your actual submission ID
submission_id = "your-submission-id"  # e.g., "2024-01-15T10:30:00.123_ABCD1234"

# Note: This is a POST request
response = requests.post(
    "https://data.minorplanetcenter.net/api/action-codes/retrieve",
    json={"label": submission_id}
)

print(f"Status code: {response.status_code}")
print(f"Response: {response.json() if response.ok else response.content}")



# Understanding Submission IDs

When you submit observations to the MPC, you receive a submission ID in the response. This ID has the format:

```
YYYY-MM-DDTHH:MM:SS.mmm_xxxxxxxx
```

Where:
- `YYYY-MM-DD` is the date
- `HH:MM:SS.mmm` is the time with milliseconds
- `xxxxxxxx` is a unique identifier

For example: `2026-01-01T00:05:07.453_0000BhCE`

# Using Tracklet IDs

If you know the tracklet submission ID (trksub) for your observations, you can use that as well.

<span style="color: red;">The response in this example will indicate a problem (because the supplied trksub,  `ABC12345` is not a known trksub in the MPC's system).</span>


In [ ]:
# Example with a tracklet ID
# Replace with your actual tracklet ID
trksub = "ABC12345"  # 1-8 alphanumeric characters

response = requests.post(
    "https://data.minorplanetcenter.net/api/action-codes/retrieve",
    json={"label": trksub}
)

print(f"Status code: {response.status_code}")
print(f"Response: {response.json() if response.ok else response.content}")



# Helper Function

Here's a convenient helper function for requesting action code retrieval.

<span style="color: red;">Again, the response in this example will indicate a problem (because the supplied submission-ID,  `your-submission-id` is not a valid submission-ID).</span>


In [ ]:
def request_action_code(label):
    """
    Request retrieval of an action code for a submission.
    
    The action code will be emailed to the original submitter's address.
    
    Parameters
    ----------
    label : str
        Submission identifier (submission ID, tracklet ID, etc.)
    
    Returns
    -------
    dict
        API response with status information
    """
    response = requests.post(
        "https://data.minorplanetcenter.net/api/action-codes/retrieve",
        json={"label": label}
    )
    
    return {
        "success": response.ok,
        "status_code": response.status_code,
        "message": "Action code will be emailed to original submitter" if response.ok else "Request failed",
        "response": response.json() if response.ok else str(response.content)
    }


# Example usage (replace with your actual submission ID)
result = request_action_code("your-submission-id")
print(json.dumps(result, indent=2))

# When to Use This API

You might need to retrieve your action code when:

1. **You've lost the original email** containing your action code
2. **You need to retract observations** that were submitted in error
3. **You want to replace observations** with corrected astrometry
4. **You're verifying submission ownership** for data management purposes

### What Action Codes Allow You To Do

With an action code, you can:
- Delete (retract) your submitted observations from the MPC database
- Replace observations with updated versions
- Prove that you are the original submitter

# Security Considerations

The Action Codes API is designed with security in mind:

1. **Action codes are never returned in API responses** - they are only sent via email
2. **Email goes to the original submitter** - only the person who submitted the observations can receive the code
3. **Action codes should be kept secret** - treat them like passwords

If someone else gains access to your action code, they could potentially retract or modify your observations. Keep your action codes secure.

# Summary

The MPC Action Codes API allows you to retrieve action codes for your MPC submissions.

Key points:
- **Endpoint**: `https://data.minorplanetcenter.net/api/action-codes/retrieve`
- **Method**: POST (not GET)
- **Required parameter**: `label` - your submission identifier
- **Output**: Action code is emailed to the original submitter (not returned in response)

Label formats accepted:
- Submission ID: `2024-01-15T10:30:00.123_ABCD1234`
- Tracklet ID: `ABC12345`
- Track ID: `ABCD12345678`
- Submission Block ID: `2024-01-15T10:30:00.123_ABCD1234_01`

For questions or feedback, contact the MPC via the [Jira Helpdesk](https://mpc-service.atlassian.net/servicedesk/customer/portal/13/create/148).